# Aula 05 - Trabalhando com Grandes Arquivos (alguns parâmetros do pd.read_csv)

- Quando estamos trabalhando com **Grandes Arquivos** temos um desafio um grande desafio que é gerenciar a memória
- As vezes precisamos manipular uma base de dados muito grande e por isso precisamos trabalhar com arquivos de forma diferente
- Uma forma é ler esses arquivos de forma limitada para não consumir toda a **memória** do servidor

In [0]:
import pandas as pd

## pd.read_csv( "path", sep = ' ' )

In [0]:
arquivo = '/content/drive/My Drive/Colab Notebooks/Data Science do ZERO/5 - Python para Análise de Dados/Materiais Disponibilizados/Pandas - Materiais de Apoio/kc_house_data.csv'
df = pd.read_csv(arquivo, sep=',')

In [3]:
# Imprimindo as 5 primeiras linhas do df
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2.0,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Parâmetro "nrows = n"

- O parâmetro "nrows" realiza a importação apenas das "n" primeiras linhas. Esse parâmtro é muito útil quando trabalha-se com bancos de dados muito grandes. As vezes é melhor carregar menos linhas apenas para visualizar o banco de dados e não comprometer a memória do servidor

In [4]:
df_min = pd.read_csv(arquivo, sep=',', nrows=10)
df_min

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


### Parâmetro "chunksize = n"

- Informa o número de linhas "n" que serão colocadas por bloco
- Isso é extremamente útil para trabalhar com Big Data
- Uma vez que o servidor processa as linhas de um bloco, a memória do servidor se esvazia para ler as linhas do próximo bloco. Isso faz com que todo o arquivo seja lido em blocos, o que diminui a memória gasta pelo servidor de uma vez
- O objeto de saída é do tipo "arquivo". É um objeto binário do Pandas
- O objeto de saída pode ser iterado

In [0]:
# No caso, como o dataset é pequeno haverá dois blocos com 10000 linhas e um com
# 1613
chunk = pd.read_csv(arquivo, chunksize=10000)

In [6]:
# É um objeto do tipo arquivo. É um objeto do pandas binário
type(chunk)

pandas.io.parsers.TextFileReader

In [7]:
# Imprimindo tamanho das partes do arquivo chunk.
for bloco in chunk:
    print (len(bloco))

10000
10000
1613


#### df [ "new_column" ] = pd.concat (list)

- Imagine que a memória do servidor é limitada para carregar o dataset inteiro de uma vez
- Nesse sentido, podemos utilizar o parâmetro "chunksize" para dividir o dataset em blocos
- Imagine que eu queira fazer uma operação com determinada coluna desse dataset gigante. Podemos para isso:
  - Criar uma lista vazia
  - Criar um laço "for" que intera sobre cada bloco do dataset "chunk"
  - O comando associado ao laço será adicionar o resultado da multiplicação de uma coluna por 2 de cada bloco à lista vazia, para cada iteração
  - Ao final, concatenaremos essa lista a um outro dataset "df" (esse outro dataset já deve ter sido declarado)

In [0]:
chunk = pd.read_csv(arquivo, chunksize=10000)

In [0]:
lista = [] # Declarando lista vazia
# Criando laço que intera sobre cada bloco do dataframe "chunk"
for bloco in chunk:
  lista.append(bloco['bedrooms'] * 2) # Repare que o bloco atua como um df

# Concatenação da lista "alimentada" ao dataset
df['bedrooms_size'] = pd.concat(lista)

In [10]:
# Visualizando o novo df com uma coluna
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_size
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,6.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,6.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,4.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,8.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,6.0


# Aula 06 - Trabalhando com Grandes Arquivos (identificando separador de elementos e quais colunas serão usadas)

1.   Tente trabalhar apenas com as colunas que você vai realmente precisar ( o Pandas não é uma biblioteca para trabalhar propriamente com Big Data. É uma biblioteca para manipulação de dados e apresenta limitações com relação a grandes bases de dados)
2.   Atente para o tipo de dado de cada coluna (isso também pode ser válido para otimizar a memória do servidor)
3.   Visualize qual o separador usado para separar os dados

## Visualizar as 5 primeiras linhas de um arquivo para identificar o separador de elementos
- Se estiver no Windows abra o prompt do **PowerShell** e use o comando abaixo:
    - **gc file_name.csv -head 5**
- Isso é útil quando queremos verificar apenas as 5 primeiras linhas de um arquivo csv muuuuuuuito grande sem travar o editor de textos. Assim é possível, por exemplo, verificar qual separador separa os elementos do arquivo
- Se estiver no Google Colab, digite o comando abaixo para obter o mesmo resultado:
    - **!head -n 5 "path"**

In [11]:
!head -n 5 '/content/drive/My Drive/Colab Notebooks/Data Science do ZERO/5 - Python para Análise de Dados/Materiais Disponibilizados/Pandas - Materiais de Apoio/kc_house_data.csv'

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
"7129300520","20141013T000000",221900,3,1,1180,5650,"1",0,0,3,7,1180,0,1955,0,"98178",47.5112,-122.257,1340,5650
"6414100192","20141209T000000",538000,3,2.25,2570,7242,"2",0,0,3,7,2170,400,1951,1991,"98125",47.721,-122.319,1690,7639
"5631500400","20150225T000000",180000,2,1,770,10000,"1",0,0,3,6,770,0,1933,0,"98028",47.7379,-122.233,2720,8062
"2487200875","20141209T000000",604000,4,3,1960,5000,"1",0,0,5,7,1050,910,1965,0,"98136",47.5208,-122.393,1360,5000


## Lendo o arquivo CSV com os parâmetros "sep" e "nrows"

- A diferença desse comando com "nrows = 5" pro comando de cima é que aqui já se sabe qual é o separador dos elementos do arquivo
- Portanto, os dois comandos têm utilidades distintas

In [0]:
import pandas as pd

In [13]:
# Ler a base com o parâmetro "nrows = 5". Importa apenas as 5 primeiras linhas
arquivo = '/content/drive/My Drive/Colab Notebooks/Data Science do ZERO/5 - Python para Análise de Dados/Materiais Disponibilizados/Pandas - Materiais de Apoio/kc_house_data.csv'
df = pd.read_csv(arquivo, sep=',', nrows=5)
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Definição de Quais Colunas Utilizar na Importação da Base de Dados

- É melhor estudar a base de dados previamente para avaliar quais colunas serão utilizadas para não trabalharmos com um df gigante com muitas colunas

### df.columns.tolist ( )

- Como já importamos as 5 primeiras linhas da base de dados, agora podemos extrair os nomes de todas as colunas para um objeto do tipo "lista"
- A partir daí, é possível escolher quais colunas iremos utilizar no projeto

In [14]:
df.columns.tolist()

['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

### Parâmetro "usecols = [col1 , col2 , ... , coln] " do pd.read_csv ( )

- Agora que já extraímos os nomes das colunas, podemos usar o parâmetro "usecols" para definir com quais colunas iremos trabalhar
- Essa seleção é feita por meio dos **nomes** das colunas

In [0]:
df = pd.read_csv(arquivo,sep = ',',
                 usecols=['id','date','price','bedrooms',
                          'bathrooms','sqft_living',
                          'sqft_lot','floors','waterfront'])

In [16]:
# Repare como o df ficou mais legível, após a seleção das colunas
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180,5650,1.0,0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570,7242,2.0,0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770,10000,1.0,0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960,5000,1.0,0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680,8080,1.0,0


# Aula 07A - Trabalhando com Grandes Arquivos

In [17]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180,5650,1.0,0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570,7242,2.0,0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770,10000,1.0,0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960,5000,1.0,0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680,8080,1.0,0


## Parâmetro "usecols=[index1 , index2 , ... , indexn] " do pd.read_csv ( )

- Seleção de colunas por índice/posição
- Antes haviamos selecionado as colunas a serem usadas pelos **nomes** com o parâmetro "usecols"
- Agora podemos selecionar as colunas pelos **índices**

In [18]:
arquivo = '/content/drive/My Drive/Colab Notebooks/Data Science do ZERO/5 - Python para Análise de Dados/Materiais Disponibilizados/Pandas - Materiais de Apoio/kc_house_data.csv'
# Utilizaremos as 6 primeiras colunas do banco de dados. A seleção é feita a
# partir dos índices das colunas
df = pd.read_csv(arquivo, usecols=[0,1,2,3,4,5])
# Visualizando o novo df
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570
2,5631500400,20150225T000000,180000.0,2.0,1.00,770
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680


## Comparação do Uso da Memória do Servidor

- Compararemos o uso da memória do servidor quando importamos o banco de dados completo e quando escolhemos apenas determinadas colunas do banco de dados para importar

### Importação do Banco de Dados Completo

- Total de 21 colunas importadas
- 3.5 MB utilizados da memória do servidor
- OBS.: no exemplo, o df é pequeno, mas na realidade, trabalha-se com df de > 1GB

In [0]:
# Importação default do banco de dados
df = pd.read_csv(arquivo, sep=',')

In [20]:
# Verificando infos gerais do df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21609 non-null  float64
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21612 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

### Importação de Parte do Banco de Dados

- Agora, o df importado conterá apenas as 6 primeiras colunas do banco de dados original
- 6 colunas importadas
- 1013.2 KB (~ 1 MB) utilizados, apenas
- Repare como uso da memória do servidor foi poupado

In [0]:
df = pd.read_csv(arquivo, usecols=[0,1,2,3,4,5])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           21613 non-null  int64  
 1   date         21613 non-null  object 
 2   price        21613 non-null  float64
 3   bedrooms     21609 non-null  float64
 4   bathrooms    21613 non-null  float64
 5   sqft_living  21613 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 1013.2+ KB


## Parâmetro "usecols = lambda..." do pd.read_csv ( )

- É utilizado para leitura de colunas de um dataset, **exceto** algumas
- Quando tem-se um dataset de 200/300 colunas, é inviável dar um **ctrl + C** no "tolist" e só queremos excluir algumas colunas da importação
- "Lambda" é uma expressão que funciona como um loop

In [0]:
arquivo = '/content/drive/My Drive/Colab Notebooks/Data Science do ZERO/5 - Python para Análise de Dados/Materiais Disponibilizados/Pandas - Materiais de Apoio/kc_house_data.csv'

In [27]:
# Lendo colunas, que não sejam (not in): ['sqft_living','sqft_lot','floors']
df = pd.read_csv(arquivo, 
                 usecols = lambda column : column not in ['sqft_living',
                                                          'sqft_lot','floors'])
df.head()

,id,date,price,bedrooms,bathrooms,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3.0,2.25,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2.0,1.00,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4.0,3.00,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3.0,2.00,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


# Aula 07 - Trabalhando com os Tipos de Dados Adequados

- É importante saber os tipos de dados de objeto que cada coluna do df apresenta
- Quando cria-se um dataframe e o algorítimo não entende qual tipo de objeto uma das colunas contém, essa coluna é assinalada como do tipo OBJECT. No entanto, o tipo "object" é aquele que mais gasta memória do servidor
- A transformação de atributos para objetos que gastam menos memória é crucial para trabalhar com grandes dados

In [0]:
# Importando base de dados "Titanic"
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

In [30]:
# Observe que algumas colunas foram reconhecidas como do tipo "objects"
# Memória do servidor gasta --> 83.7 KB
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## df.attribute = df.attribute.astype ( "category" )

- Tranforma uma coluna do dataframe em uma coluna de objetos do tipo categórico (gasta menos memória)

In [0]:
df.Sex = df.Sex.astype('category')
df.Embarked = df.Embarked.astype('category')
df.Survived = df.Survived.astype('category')
df.Pclass = df.Pclass.astype('category')

## df.feature = df.feature.astype ( "int32" )

- Tranforma uma coluna do dataframe em uma coluna de objetos do tipo inteiro (gasta menos memória)
- Tranformamos objetos do tipo "int64" para objetos do tipo "int32". Como o atributo é numérico, mas sabe-se que não apresenta muitos caracteres, podemos fazer isso para poupar memória

In [0]:
df.PassengerId = df.PassengerId.astype('int32')
df.Parch = df.Parch.astype('int32')
df.SibSp = df.SibSp.astype('int32')

In [35]:
# Note como o consumo da memória do servidor reduziu com a tranformação dos
# objetos do tipo "object" para "categorical" e "int32"
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int32   
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    category
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int32   
 7   Parch        891 non-null    int32   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(4), float64(2), int32(3), object(3)
memory usage: 49.2+ KB


## Parâmetro "dtype = {"col_name" : "type", ... } "

- Serve para já importar determinadas colunas já em determinado tipo de objeto
- O parâmetro "dtype" recebe um dicionário, cuja chave é o nome da coluna e o valor é o tipo de objeto da coluna

In [0]:
# Importando determinadas colunas já definindo os tipos de objeto de cada uma
data = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(data,  dtype = {"Embarked" : "category", 
                                 "Survived": "category", "Parch": "int32"})

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    object  
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int32   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(2), float64(2), int32(1), int64(3), object(4)
memory usage: 68.1+ KB
